### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

In [2]:
from utils.factorization import SVD, ALS

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [5]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
# model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
# model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
# movie_info.head(5)

In [12]:
# get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
#                                         for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
# get_similars(1, model)

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
# get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
#                                             for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
# get_user_history(4, implicit_ratings)

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
# get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
#                                                for x in model.recommend(user_id, user_item_csr)]

In [17]:
# get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [18]:
def test_model(model, test_id=1, user_id=4, n=10):
    target = movie_info[movie_info.movie_id == test_id]
    sims_indxs = model.get_similar_items_indxs(test_id, n)

    print(f'For {target.name.item()} ({target.category.item()}) found these {n} similiars):')
    for indx in sims_indxs:
            similar_item = movie_info[movie_info.movie_id == indx]
            if len(similar_item) == 0:
                print("movie doesn't exist")
            else:
                print(f'\t {similar_item.name.item()} ({similar_item.category.item()})')
            
    print('____________________________________________')

    recs = model.recommend(user_id, n)
    print(f"For user {user_id} found these movies:")
    for idx in recs:
            rec = movie_info[movie_info.movie_id == idx]
            if len(rec) == 0:
                print("movie doesn't exist")
            else:
                print(f'\t {rec.name.item()} ({rec.category.item()})')

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [19]:
user_item_exp = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

In [20]:
model_svd = SVD(64, max_iters=500, lr=1e-4, verbose=100, lam=1e-3)
model_svd.fit(user_item_exp);

Iter: 100, score: 0.9353555103849236
Iter: 200, score: 0.9072005915293382
Iter: 300, score: 0.8921807185645498
Iter: 400, score: 0.8649124969813241
Iter: 500, score: 0.8328136357699623


In [21]:
test_model(model_svd)

For Toy Story (1995) (Animation|Children's|Comedy) found these 10 similiars):
	 Toy Story (1995) (Animation|Children's|Comedy)
	 Money Train (1995) (Action)
	 City of Lost Children, The (1995) (Adventure|Sci-Fi)
	 Cutthroat Island (1995) (Action|Adventure|Romance)
	 Father of the Bride Part II (1995) (Comedy)
	 Eye for an Eye (1996) (Drama|Thriller)
	 Dracula: Dead and Loving It (1995) (Comedy|Horror)
	 GoldenEye (1995) (Action|Adventure|Thriller)
	 Get Shorty (1995) (Action|Comedy|Drama)
	 Big Green, The (1995) (Children's|Comedy)
____________________________________________
For user 4 found these movies:
	 Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) (Film-Noir)
	 General, The (1927) (Comedy)
	 Close Shave, A (1995) (Animation|Comedy|Thriller)
	 Wrong Trousers, The (1993) (Animation|Comedy)
	 Godfather, The (1972) (Action|Crime|Drama)
	 Schindler's List (1993) (Drama|War)
	 Paths of Glory (1957) (Drama|War)
	 Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) (Actio

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [22]:
model_als = ALS(64, max_iters=15, lr=1e-4, verbose=2, lam=1e-5)
model_als.fit(user_item_csr)

test_model(model_als)

Iter: 2, score: 0.9228335947715938
Iter: 4, score: 0.923234296826092
Iter: 6, score: 0.9236397757624473
Iter: 8, score: 0.9238625495785319
Iter: 10, score: 0.9239965972134925
Iter: 12, score: 0.9240836337409102
Iter: 14, score: 0.9241436647561159
For Toy Story (1995) (Animation|Children's|Comedy) found these 10 similiars):
	 Toy Story (1995) (Animation|Children's|Comedy)
	 Sabrina (1995) (Comedy|Romance)
	 Restoration (1995) (Drama)
	 Wings of Courage (1995) (Adventure|Romance)
	 Mighty Aphrodite (1995) (Comedy)
	 Dead Man Walking (1995) (Drama)
movie doesn't exist
	 Carrington (1995) (Drama|Romance)
	 Now and Then (1995) (Drama)
	 Dead Presidents (1995) (Action|Crime|Drama)
____________________________________________
For user 4 found these movies:
	 Phantasm (1979) (Horror|Sci-Fi)
	 Jumpin' Jack Flash (1986) (Action|Comedy|Romance|Thriller)
	 Jaws (1975) (Action|Horror)
	 Broadway Melody, The (1929) (Musical)
	 Nurse Betty (2000) (Comedy|Thriller)
	 Clueless (1995) (Comedy|Romance)
	

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных